In [ ]:
import cv2
import os
import time
from ultralytics import YOLO
from simple_facerec import SimpleFacerec

# Parameters
confidence = 0.6  # Confidence threshold for antispoofing
classNames = ["fake", "real"]  # Antispoofing classes

# Face Recognition Setup
os.chdir(r"E:/engeneering/graduation project/project/python/access control")
sfr = SimpleFacerec()
sfr.load_encoding_images("images/")  # Load known faces

# Antispoofing Model Setup
antispoofing_model = YOLO("best.pt")

# Camera setup
cap = cv2.VideoCapture(0)
cap.set(3, 640)  # Set width
cap.set(4, 480)  # Set height

# FPS calculation
prev_frame_time = 0
new_frame_time = 0

while True:
    new_frame_time = time.time()
    success, frame = cap.read()
    if not success:
        print("Failed to read from the camera. Exiting...")
        break

    # Antispoofing Detection
    spoof_results = antispoofing_model(frame, stream=True, verbose=False)
    real_faces = []  # Store real faces after antispoofing check

    for r in spoof_results:
        boxes = r.boxes
        for box in boxes:
            # Extract bounding box coordinates
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            w, h = x2 - x1, y2 - y1

            # Extract confidence and class index
            conf = round(float(box.conf[0]), 2)
            cls = int(box.cls[0])

            if conf > confidence and classNames[cls] == "real":
                # If real face, save coordinates for face recognition
                real_faces.append((x1, y1, x2, y2))
                color = (0, 255, 0)  # Green for real
            else:
                color = (0, 0, 255)  # Red for fake

            # Draw bounding box for antispoofing
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

            # Add label with confidence score
            label = f"{classNames[cls].upper()} {int(conf * 100)}%"
            text_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
            text_x = x1
            text_y = max(35, y1 - 10)

            # Draw background for text
            cv2.rectangle(frame, (text_x, text_y - text_size[1] - 5),
                          (text_x + text_size[0] + 10, text_y + 5), color, -1)
            cv2.putText(frame, label, (text_x + 5, text_y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Face Recognition for Real Faces
    for (x1, y1, x2, y2) in real_faces:
        face_region = frame[y1:y2, x1:x2]  # Crop the face region
        face_locations, face_names = sfr.detect_known_faces(face_region)

        for face_loc, name in zip(face_locations, face_names):
            # Adjust face location to original frame coordinates
            y1, x2, y2, x1 = face_loc[0] + y1, face_loc[1] + x1, face_loc[2] + y1, face_loc[3] + x1

            # Draw name and rectangle for recognized faces
            cv2.putText(frame, name, (x1, y1 - 10), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 255, 0), 2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 4)

    # Calculate FPS
    fps = int(1 / (new_frame_time - prev_frame_time))
    prev_frame_time = new_frame_time
    cv2.putText(frame, f"FPS: {fps}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    # Display the output
    cv2.imshow("Face Recognition with Antispoofing", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
